# Reading and writing sequences with BioPython
(Víctor Sojo | vsojo@amnh.org)

In the previous lesson we had our first experience interacting with NCBI through BioPython. We downloaded `4` records that seem to contain the DNA sequence for the gene encoding the 12S rRNA of the red panda.

In this notebook we will continue our **BioPython** training by:
1. Reading back in the records that we downloaded in the previous lesson.
1. Extracting the `features` that contain the sequences of the 12S rRNA gene.
1. Assembling the sequences of the four files into a common FASTA file, which we will use later in the alignment lesson.

**References & recommended reading:**
+ The [_BioPython tutorial_](http://biopython.org/DIST/docs/tutorial/Tutorial.html).
+ Tiago Antao's [_Bioinformatics with Python Cookbook_](https://www.packtpub.com/product/bioinformatics-with-python-cookbook-second-edition/9781789344691).

## Contents
&emsp;[Importing required BioPython modules](#Importing-required-BioPython-modules)<br/>
&emsp;[Re-explore the file with the list of GenBank files](#Re-explore-the-file-with-the-list-of-GenBank-files)<br/>
&emsp;[Reading single-sequence files using BioPython's SeqIO.read\(\) ](#Reading-single-sequence-files-using-BioPython's-SeqIO.read\(\)-)<br/>
&emsp;[Extracting features and their sequences in a BioPython record](#Extracting-features-and-their-sequences-in-a-BioPython-record)<br/>
&emsp;[Creating new records and writing them to a file](#Creating-new-records-and-writing-them-to-a-file)<br/>
&emsp;[Reading a file with multiple sequences using SeqIO.parse\(\)](#Reading-a-file-with-multiple-sequences-using-SeqIO.parse\(\))<br/>
&emsp;&emsp;[Should I use SeqIO.parse\(\) or SeqIO.read\(\) to load my sequence file?](#Should-I-use-SeqIO.parse\(\)-or-SeqIO.read\(\)-to-load-my-sequence-file?)<br/>

Again, let's make sure that we're using the `bioinfo` environment that we created in the `Py201` notebook:

In [1]:
!echo $CONDA_DEFAULT_ENV

bioinfo


(If you're on Windows, remember that every line with `!my code` should be changed to `!wsl my code` and you should have an active [WSL installation](https://docs.microsoft.com/en-us/windows/wsl/install-win10))

## Importing required BioPython modules
Here we will need:

Module      | Use
:-----------|:-----------------------------------------
**Bio.SeqIO**   | To handle parsing, reading and writing sequences
**Bio.SeqRecord.SeqRecord** | To create new sequence records

In [2]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

## Re-explore the file with the list of GenBank files
In the previous lesson, we used **Entrez** to download data from NCBI. We found `4` records, and stored each of them into a GenBank file. Cleverly, we also stored the names of the files that we created into a file of their own, so that we could automate our analyses. This file simply contains the location of names of the files. We will use that file here so that we can read the sequence records back in.

First, let's just take a look at the file:

In [3]:
gb_files_file = 'gb_files.list'
!cat $gb_files_file

GenBank/NC_011124.1.gb
GenBank/AM711897.1.gb
GenBank/Y08511.1.gb
GenBank/L21885.1.gb


That looks good. We can use this list file to open each of the GenBank files.

There are 4 files, which makes sense since we had 4 gene IDs.

Once again, let's take a look at the first few lines of one of these files:

In [4]:
!head -20 GenBank/NC_011124.1.gb

LOCUS       NC_011124              16493 bp    DNA     circular MAM 14-APR-2009
DEFINITION  Ailurus fulgens mitochondrion, complete genome.
ACCESSION   NC_011124
VERSION     NC_011124.1
DBLINK      Project: 30903
            BioProject: PRJNA30903
KEYWORDS    RefSeq.
SOURCE      mitochondrion Ailurus fulgens (lesser panda)
  ORGANISM  Ailurus fulgens
            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;
            Mammalia; Eutheria; Laurasiatheria; Carnivora; Caniformia;
            Ailuridae; Ailurus.
REFERENCE   1
  AUTHORS   Arnason,U., Gullberg,A., Janke,A. and Kullberg,M.
  TITLE     Mitogenomic analyses of caniform relationships
  JOURNAL   Mol. Phylogenet. Evol. 45 (3), 863-874 (2007)
   PUBMED   17919938
REFERENCE   2  (bases 1 to 16493)
  CONSRTM   NCBI Genome Project
  TITLE     Direct Submission


Go ahead and take a look at the full file in a text editor such as Notepad++ (Win) or Sublime/BBEdit (Mac), or you can use the Jupyter browser itself.

## Reading single-sequence files using BioPython's `SeqIO.read()` 
Take a look at the second line of the fragment that we just printed above. You'll see that this particular file has the _complete genome_ of the mitochondrial chromosome of the red panda. We don't want the whole chromosome – just the 12S rRNA gene. BioPython makes it easy to extract it.

Here is our plan of action for the next block of code:
1. We will read in the file with the list of GenBank file names.
1. We will use these file names to load the GenBank files one by one.
1. For each of these files, we will turn their contents into a BioPython record using `Bio.SeqIO.read()`.
1. We will then store each of these records into a list that we will call `seqrecs`, so that we can access them later.

About that last step, we can do this here because it's only 4 records, but in typical bioinformatics work you should avoid lists of sequence records as much as possible (further details below).

In [5]:
seqrecs = [] # an empty list that will hold the sequences records

# Open the file that contains the list of GenBank files
with open(gb_files_file, 'r') as f:
    for gb_filename in f:
        gb_filename = gb_filename.strip() # Remove the \n at the end of each line, always necessary in Python
        
        # Read this file into a BioPython record
        with open(gb_filename, 'r') as g: # can't use f, we're using it in the upper loop
            # Read the file into a BioPython sequence record
            rec = SeqIO.read(g, 'gb') # We specify 'gb' because it's a GenBank file.
            print(f"Loaded record {rec.id}, whose description is:\t{rec.description}")
            # Store that record into the list (VERY BAD IDEA if you have huge data)
            seqrecs.append(rec)

Loaded record NC_011124.1, whose description is:	Ailurus fulgens mitochondrion, complete genome
Loaded record AM711897.1, whose description is:	Ailurus fulgens complete mitochondrial genome
Loaded record Y08511.1, whose description is:	A.fulgens mitochondrial 12S rRNA gene
Loaded record L21885.1, whose description is:	Ailurus fulgens mitochondrial 12S ribosomal RNA (12S rRNA) gene fragment


We now have our four sequence records in the `seqrecs` list.

⚠️ It is typically a bad idea to store records into a `list` as we're doing here. Instead, we should process them one by one in their entirety if possible. Here, I wanted to keep the steps separate from each other for educational purposes, so the list method works since it's only 4 records. However, in real bioinformatics research, you should aim for a single `for` loop in which everything happens, one record at a time. That way, we would release the memory at the end of each iteration, loading only one record into memory at any given time. With the `list` method, we're holding everything in memory simultaneously – a terrible idea for big data.

## Extracting `features` and their sequences in a BioPython record
As you can see from the `description` of each of the records that we printed above, a couple of those records contain sequences for the entire mitochondrial genome. We don't want the whole mitochondrial chromosome – just the 12S rRNA gene – so we need to extract that information from the GenBank record.

I strongly recommend that you open the file `AM711897.1.gb` in a text editor and take a look at the whole text. In brief, the file contains only one sequence (at the end), but multiple "features", one for each significant portion of DNA in the chromosome (genes, tRNAs, rRNAs and so on). In our case, we're interested in the `12S rRNA` gene, so we can look for that.

Before we do that with BioPython, let's use the advantages of Jupyter's access to bash and take a quick look with `grep`:

In [6]:
!grep -B1 -A4 "12S rRNA" GenBank/AM711897.1.gb
# -A4 means "show me 4 extra lines After the one I'm looking for".
# And you can guess what -B1 means.

VERSION     AM711897.1
KEYWORDS    12S ribosomal RNA; 12S rRNA gene; 16S ribosomal RNA; 16S rRNA gene;
            ATPase subunit 6; ATPase subunit 8; ATPase6 gene; ATPase8 gene; COI
            gene; COII gene; COIII gene; cytb gene; cytochrome b; cytochrome
            oxidase subunit I; cytochrome oxidase subunit II; cytochrome oxidase
            subunit III; NADH dehydrogenase subunit 1; NADH dehydrogenase
--
--
     gene            70..1034
                     /gene="12S rRNA"
     rRNA            70..1034
--
     rRNA            70..1034
                     /gene="12S rRNA"
                     /product="12S ribosomal RNA"
     gene            1035..1102
                     /gene="tRNA-Val"
     tRNA            1035..1102


You'll see on the left that there are features of several types (`gene`, `rRNA`, `tRNA`). We are looking for a `gene` encoding the `12S rRNA`.

You'll notice also that there's a `gene` encoding a `tRNA` for valine at positions `1035..1102`. Before that one, you'll see our `gene` of interest, which is located at `70..1034`. We can extract it programmatically using BioPython, through the `features` property for each of the 4 records. This will let us filter for only those features classified as a `gene` with `12S rRNA` in the name.

As a first step to get familirised with the syntax, let's extract and print our desired features:

In [7]:
# Go over each of the four records:
for rec in seqrecs:
    print(f"\nProcessing record {rec.id}, whose full sequence is:\t{rec.seq[:10]}...{rec.seq[-3:]}")
    for feature in rec.features:
        if feature.type == 'gene' and '12S rRNA' in feature.qualifiers['gene']:
            print(f"\tFound a 12S rRNA gene located at {feature.location}")
            print(f"\tThe 12S rRNA-encoding gene has a sequence of {feature.extract(rec.seq)[:10]}...")


Processing record NC_011124.1, whose full sequence is:	GTTAATGTAG...TCT
	Found a 12S rRNA gene located at [69:1034](+)
	The 12S rRNA-encoding gene has a sequence of CCAAAGGTTT...

Processing record AM711897.1, whose full sequence is:	GTTAATGTAG...TCT
	Found a 12S rRNA gene located at [69:1034](+)
	The 12S rRNA-encoding gene has a sequence of CCAAAGGTTT...

Processing record Y08511.1, whose full sequence is:	CAAAGGTTTG...AAC
	Found a 12S rRNA gene located at [<0:>964](+)
	The 12S rRNA-encoding gene has a sequence of CAAAGGTTTG...

Processing record L21885.1, whose full sequence is:	AAATAGTTTA...GAG
	Found a 12S rRNA gene located at [0:349](+)
	The 12S rRNA-encoding gene has a sequence of AAATAGTTTA...


That was great. But here we just printed a bit of the information out. What we'd really like to do is save each of those sequences to a common FASTA file, so that we can compare them and check if they're the same (at least one of them won't be, since it's much shorter than the rest). Let's do that next.

## Creating new records and writing them to a file
Let's modify our last bit of code slightly so that, instead of just printing out that we found our desired 12S rRNA gene, we actually create a new sequence record with just that relevant bit and put it out to a common FASTA file.

(Note: We are calling this file `unaligned` because the sequences have no positional reference to one another at this point. We will align them in the next lesson)

In [8]:
# The name of our fasta file:
multi_fasta_file = "RedPanda_12S_rRNA.unaligned.fasta"

# Create the file empty to begin with:
with open(multi_fasta_file, 'w') as f:
    pass # 'pass' is Python jargon for "do nothing"... we just want to create an empty file

# Go over each of the four records:
for rec in seqrecs:
    print(f"Processing record {rec.id}, whose full sequence is:\t{rec.seq[:10]}...{rec.seq[-3:]}")
    for feature in rec.features:
        if feature.type == 'gene' and '12S rRNA' in feature.qualifiers['gene']:
            print("\tFound a 12S rRNA gene located at {feature.location}")
            print("\t...extracting it now and putting it into a common FASTA")
            # First we need to create a new record for this fragment of sequence:
            rec_12S = SeqRecord(
               feature.extract(rec.seq) # extract the sequence corresponding to this feature
              ,id = rec.id + '.12SrRNA'   # Use the id for each record as the base for the sequence identifier
              ,description = f"12S rRNA gene for Red Panda (Ailurus fulgens) record {rec.id}"
            )
            
            # Append this record to a common file, in FASTA format:
            with open(multi_fasta_file, 'a') as f:
                f.write(rec_12S.format('fasta'))
            
            print("\tDone!\n")

Processing record NC_011124.1, whose full sequence is:	GTTAATGTAG...TCT
	Found a 12S rRNA gene located at {feature.location}
	...extracting it now and putting it into a common FASTA
	Done!

Processing record AM711897.1, whose full sequence is:	GTTAATGTAG...TCT
	Found a 12S rRNA gene located at {feature.location}
	...extracting it now and putting it into a common FASTA
	Done!

Processing record Y08511.1, whose full sequence is:	CAAAGGTTTG...AAC
	Found a 12S rRNA gene located at {feature.location}
	...extracting it now and putting it into a common FASTA
	Done!

Processing record L21885.1, whose full sequence is:	AAATAGTTTA...GAG
	Found a 12S rRNA gene located at {feature.location}
	...extracting it now and putting it into a common FASTA
	Done!



Good! Let's take a look at the full file:

In [9]:
!cat $multi_fasta_file

>NC_011124.1.12SrRNA 12S rRNA gene for Red Panda (Ailurus fulgens) record NC_011124.1
CCAAAGGTTTGGTCCTAGCCTTCCCGTTAGTTCTTAATAAAATTACACATGCAAGTATCT
ACACCCCAGTGAAAATGCCCTCCAAATCACTAGTTGATTAAAAGGAGCAGGTATCAAGCA
CACTTTACATAAGTAGCTCACAACACCTTGCTTAGCCACACCCCCACGGGATACAGCAGT
GATAAAAATTAAGCTATAAACGAAAGTTTGACTAAGTTATATTAATAGAGGGTTGGTAAA
TTTCGTGCCAGCCACCGCGGCCATACGATTAACCCAAACTAACGGGCATCCGGCGTAAAA
CGTGTTAAAGAATCTACTCACATTAAAGTTAAAACTTAATCAGGCCGTAAAAAGCTACCG
TTAACATAAAATACCTTACGAAAGTAACTTTATTAATTCTGATTACACGACAGCTAAGGC
CCAAACTGGGATTAGATACCCCACTATGCTTAGCCCTAAACATAAATAGTTTAGTATAAC
AAAACTATTCGCCAGAGAACTACTAGCAATAGCCTAAAACTCAAAGGACTTGGCGGTGCT
TTACACCCCTCTAGAGGAGCCTGTTCTATAATCGATAAACCCCGATAAACCTTACCACTT
CTAGCTACTTCAGTCTATATACCGCCATCTTCAGCAAACCCTCAAAAGGAAGCAAAGTAA
GCATAATGATTCCTGCATAAAAAAGTTAGGTCAAGGTGTAACCCATGAAGTGGAAAGAAA
TGGGCTACATTTTCTAAACAAGAACACTATACGAAAATTTTTATGAAATTAAAACCTAAA
GGTGGATTTAGTAGTAAATTAAGAATAGAGAGCTTAGTTGAATTGGGCTATAAAGCACGC
ACACACCGCCCGTCACCCTCCTCAAACGTCAAGATTTAAACCCAATATATAATCACAACA

---
⚠️This file was short, so it was ok to print it all to screen with `cat`. That's often not the case in bioinformatics and you should instead use `grep` or `head`.

## Reading a file with multiple sequences using `SeqIO.parse()`
We don't need to, but just so we know how to do it, let's read the file back in, using BioPython.

⚠️ Above, when we read the GenBank files that had a _single sequence_, we used **`SeqIO.read()`**. Here, since we're reading a file with _multiple sequences_, we instead use **`SeqIO.parse()`**.

In [10]:
seqrecs = SeqIO.parse(multi_fasta_file, 'fasta')
print(seqrecs)

So, `SeqIO.parse()` gives us an iterator, in which each item corresponds to each of the independent sequences in the file, in this case our 4 sequences.

In [11]:
for rec in seqrecs:
    print(rec)
    print(50*"% " + "\n") # Just a nice separator

ID: NC_011124.1.12SrRNA
Name: NC_011124.1.12SrRNA
Description: NC_011124.1.12SrRNA 12S rRNA gene for Red Panda (Ailurus fulgens) record NC_011124.1
Number of features: 0
Seq('CCAAAGGTTTGGTCCTAGCCTTCCCGTTAGTTCTTAATAAAATTACACATGCAA...AAC')
% % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % 

ID: AM711897.1.12SrRNA
Name: AM711897.1.12SrRNA
Description: AM711897.1.12SrRNA 12S rRNA gene for Red Panda (Ailurus fulgens) record AM711897.1
Number of features: 0
Seq('CCAAAGGTTTGGTCCTAGCCTTCCCGTTAGTTCTTAATAAAATTACACATGCAA...AAC')
% % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % 

ID: Y08511.1.12SrRNA
Name: Y08511.1.12SrRNA
Description: Y08511.1.12SrRNA 12S rRNA gene for Red Panda (Ailurus fulgens) record Y08511.1
Number of features: 0
Seq('CAAAGGTTTGGTCCTAGCCTTCCCGTTAGTTCTTAATAAAATTACACATGCAAG...AAC')
% % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % % 

You'll see that BioPython nicely summarises the sequence and prints some useful information. This is the information that we gave when we created the FASTA file above.

Note: If you try to rerun the last `for` loop, you'll get nothing, because iterators run only once, as you surely know by now 😉. You'd have to go one further cell up to regenerate the iterator.

### Should I use `SeqIO.parse()` or `SeqIO.read()` to load my sequence file?
+ Use **`SeqIO.read()`** to load files that contain **only one sequence** (e.g., a single `>` in the case of a FASTA file).
+ Use **`SeqIO.parse()`** to load files with **multiple sequences** (multiple `>` headers in the case of a FASTA file).

Note that a GenBank file may have multiple `feature`, but it has only one _sequence_, so you should read it with `read()` as we did above.

Note also that `.parse()` returns an `iterator`, in which each item is a record that corresponds to each of the sequences in the original file.<br/>
Conversely, `.read()` returns a BioPython sequence record directly, so you should only use it if you have a single sequence in the file.

In the next lesson, we will work with sequence alignments. As a starter, we will reload the FASTA file that we made here to contain the 4 red panda 12S rRNA sequences. We will then align it, and take a look at the alignment.